# Motherhood in the workfoce
This notebook contains steps I used to clean my data obtained from IPUMS HigherED.

In [21]:
#import packages
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('whitegrid')
sns.set_context("talk")
%matplotlib inline 

In [22]:
#read file
#after initial read, changed known null values to NaN
df = pd.read_csv('highered_00005.csv', na_values=[96, 99, 98, 9996, 9998, 9999, 999996, 999998, 999999, 9999998, 9999999])

In [23]:
#explore dataframe
df.head()

,PERSONID,YEAR,WEIGHT,SAMPLE,SURID,AGE,GENDER,RACETH,CHTOT,DGRDG,LFSTAT,EMSEC,EMSIZE,NEWBUS,WAPRSM,SUPWK,SALARY,CHFAM
0,50000030000000003,2003,372.6047,601,1,47,1,2,1.0,1,1,1.0,3.0,0.0,2.0,1.0,35000.0,NaN
1,50000030000000004,2003,341.7484,601,1,66,2,2,NaN,1,1,4.0,1.0,0.0,1.0,1.0,66000.0,NaN
2,50000030000000005,2003,522.5629,601,1,60,1,2,NaN,1,1,4.0,6.0,0.0,3.0,1.0,102000.0,NaN
3,50000030000000006,2003,135.0212,601,1,30,2,2,NaN,2,1,4.0,8.0,0.0,1.0,1.0,81000.0,NaN
4,50000030000000104,2003,907.4463,601,1,68,1,2,NaN,2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#only column upon initial read with NaN values. Explore values to determine breakdown.
#0=NO, 1=Yes, 98=Logical skip
df['CHFAM'].value_counts()

98.0    89438
0.0     22250
1.0      3464
Name: CHFAM, dtype: int64

In [7]:
df['CHTOT'].value_counts()

98    113521
3      56237
1      36945
Name: CHTOT, dtype: int64

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206703 entries, 0 to 206702
Data columns (total 18 columns):
PERSONID    206703 non-null int64
YEAR        206703 non-null int64
WEIGHT      206693 non-null float64
SAMPLE      206703 non-null int64
SURID       206703 non-null int64
AGE         206703 non-null int64
GENDER      206703 non-null int64
RACETH      206703 non-null int64
CHTOT       93182 non-null float64
DGRDG       206703 non-null int64
LFSTAT      206703 non-null int64
EMSEC       176857 non-null float64
EMSIZE      176857 non-null float64
NEWBUS      176857 non-null float64
WAPRSM      176857 non-null float64
SUPWK       176857 non-null float64
SALARY      176857 non-null float64
CHFAM       25714 non-null float64
dtypes: float64(9), int64(9)
memory usage: 28.4 MB


Initial read: all columns have the same entries of 206703, except CHFAM, which has 115152 entries. Viewing the first 5 rows of data shows that this data is in a highly engineered format where values represent null values, i.e. na_values=[96, 99, 98, 9996, 9998, 9999, 999996, 999998, 999999, 9999998, 9999999] This is added onto the initial data read.

Now, we see that the total number of entries is 206703, which is also true for most of columns except CHTOT, EMSEC, EMSIZE, NEWBUS, SUPWK, SALARY, AND CHFAM.
We know from initial read that CHTOT had full entries with non-null values until these values were switched to null. Since they became null due to logical skips, we can assume that these individuals do not have any children. All null values will be changed to 0 = no children.

In [25]:
#change null-values in CHTOT to 0, corresponding to no.
df['CHTOT'] = df['CHTOT'].fillna(0)

Notice that all the columns pertaining to employment have the value of 176857. First assumption is that this is due to employment status.

In [26]:
#view total employment status, 1=employed, 2=unemployed, 3=not in labor force
df['LFSTAT'].value_counts()

1    176857
3     23958
2      5888
Name: LFSTAT, dtype: int64

We see above that the total employed status is 176857 which is the same as the above columns. Now we know why those values were null and will leave them as is.

Since CHFAM column is a question worded specifically for termination of employment due to family related reasons, we will assume a default answer of no. The previous info() method shows that there are 25714 non null values which is approximately the same value as unemployment. All null values will be converted to 0 and the column will be left as it is. 

In [27]:
#change null-values to 0 = No
df['CHFAM'] = df['CHFAM'].fillna(0)

In [29]:
#check for missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206703 entries, 0 to 206702
Data columns (total 18 columns):
PERSONID    206703 non-null int64
YEAR        206703 non-null int64
WEIGHT      206693 non-null float64
SAMPLE      206703 non-null int64
SURID       206703 non-null int64
AGE         206703 non-null int64
GENDER      206703 non-null int64
RACETH      206703 non-null int64
CHTOT       206703 non-null float64
DGRDG       206703 non-null int64
LFSTAT      206703 non-null int64
EMSEC       176857 non-null float64
EMSIZE      176857 non-null float64
NEWBUS      176857 non-null float64
WAPRSM      176857 non-null float64
SUPWK       176857 non-null float64
SALARY      176857 non-null float64
CHFAM       206703 non-null float64
dtypes: float64(9), int64(9)
memory usage: 28.4 MB


Data is cleaned and ready

In [30]:
#use string categorical data to replace numerical data
gen_map = {1:'Female', 2:'Male'}
df['Genders'] = df['GENDER'].map(gen_map)

race_map = {1:'Asian', 2:'White', 3:'other minorities', 4:'Other'}
df['Race'] = df['RACETH'].map(race_map)

chi_map = {0:'none', 1:'1', 2:'1-3', 3:'2 or more', 4:'more than 3'}
df['Children'] = df['CHTOT'].map(chi_map)

deg_map = {1:'Bachelors',2:'Masters', 3:'Doctorate', 4:'Professional'}
df['Degree'] = df['DGRDG'].map(deg_map)

LF_map = {1: 'Employed', 2: 'Unemployed'}
df['LFSTATS'] = df['LFSTAT'].map(LF_map)
df['LFSTATS'] = df['LFSTATS'].replace(3,np.NaN)

ES_map = {1:'2 yr college',2:'4 yr college',3:'Government',4:'Industry',5:'Non_US government'}
df['EMSECS'] = df['EMSEC'].map(ES_map)

esize_map = {1:'Less than 10',2:'11-24',3:'25-99', 4:'100-499', 5:'500-999',6:'1000-4999',7:'5000-24999',8:'more than 25000'}
df['EMSIZES'] = df['EMSIZE'].map(esize_map)

new_map = {0:'NO', 1:'YES'}
df['NEWBUSS'] = df['NEWBUS'].map(new_map)

work_map = {1:'RND',2:'Teaching',3:'Admin',4:'Computer apps', 5:'Other'}
df['Work'] = df['WAPRSM'].map(work_map)

sup_map = {0:'No',1:'Yes'}
df['Supervisory'] = df['SUPWK'].map(sup_map)

change_map = {0:'No',1:'Yes'}
df['Family'] = df['CHFAM'].map(change_map)

In [31]:
#drop irrelevant columns
df = df.drop(['WEIGHT','SAMPLE', 'GENDER', 'RACETH', 'CHTOT', 'DGRDG', 'LFSTAT', 'EMSEC', 'EMSIZE', 'NEWBUS', 'WAPRSM', 'SUPWK', 'CHFAM'], axis=1)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206703 entries, 0 to 206702
Data columns (total 16 columns):
PERSONID       206703 non-null int64
YEAR           206703 non-null int64
SURID          206703 non-null int64
AGE            206703 non-null int64
SALARY         176857 non-null float64
Genders        206703 non-null object
Race           206703 non-null object
Children       206703 non-null object
Degree         206703 non-null object
LFSTATS        182745 non-null object
EMSECS         176857 non-null object
EMSIZES        176857 non-null object
NEWBUSS        176857 non-null object
Work           176857 non-null object
Supervisory    176857 non-null object
Family         206703 non-null object
dtypes: float64(1), int64(4), object(11)
memory usage: 25.2+ MB


In [33]:
#a dataframe for 2003 survey only
df_2003 = df[df['YEAR'] == 2003]

#a datafram for 2013 survey only
df_2013 = df[df['YEAR'] == 2013]